# Spel zonder Spelelement

Ontwikkeld met versies:
- Python 3.11.6
- Pandas 2.0.3
- ipywidgets 8.1.2

In [ ]:
# Imports
import pandas as pd
import random
import itertools
import ipywidgets as widgets

## Class SpelZonderSpelelement

In [ ]:
class SpelZonderSpelelement:
	DEFAULT_NAMES = ['Mawkmawk', 'Rap', 'Irun', 'Kurwa', 'Gerbood']

	def __init__(self, seed=None, player_names=DEFAULT_NAMES):
		self.seed = seed
		self.player_names = player_names
		self.num_players = len(self.player_names)
		self.deck = self.create_deck()
		self.player_hands = self.deal_cards()
		self.played_cards = []

	def create_deck(self):
		random.seed(self.seed)
		ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
		suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
		return list(itertools.product(ranks, suits))

	def deal_cards(self):
		random.shuffle(self.deck)
		hands = [[] for _ in range(self.num_players)]

		while self.deck:
			for i in range(self.num_players):
				if self.deck:
					card = self.deck.pop()
					hands[i].append(card)
				else:
					break
		return hands
	
	def show_hands(self, number_of_cards=12):
		return pd.DataFrame(self.player_hands, index=self.player_names).iloc[:, :number_of_cards+1]

	def play_card(self, print_output=True):
		max_card_index = []
		max_rank = None
		player_name = None
		for i in range(self.num_players):
			# Check if player has any cards left
			if self.player_hands[i]:
				card = self.player_hands[i].pop(0)
				self.played_cards.append(card)
				if print_output:
					print(f"{self.player_names[i]} played {card[0]} of {card[1]}")
				rank_value = self.get_rank_value(card[0])
				if max_rank is None or rank_value > max_rank:
					max_rank = rank_value
					max_card_index = [i]
					player_name = self.player_names[i]
				elif rank_value == max_rank:
					max_card_index.append(i)

		if len(max_card_index) > 1:
			if print_output:
				print("It's a draw! Drawing another card...")
			for i in max_card_index:
				if self.player_hands[i]:
					new_card = self.player_hands[i].pop(0)
					if print_output:
						print(f"{self.player_names[i]} draws another card: {new_card[0]} of {new_card[1]}")
					self.played_cards.append(new_card)
					rank_value = self.get_rank_value(new_card[0])
					if rank_value > max_rank:
						max_rank = rank_value
						player_name = self.player_names[i]

		if print_output:
			print(f"{player_name} wins! He receives {len(self.played_cards)} cards in his deck.")

		winning_player_index = self.player_names.index(player_name)
		self.player_hands[winning_player_index].extend(self.played_cards)
		self.played_cards = []

	def get_rank_value(self, rank):
		ranks_values = {'2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, 'J': 11, 'Q': 12, 'K': 13, 'A': 14}
		return ranks_values[rank]

## Voorbeelden

In [ ]:
# Example usage:
game = SpelZonderSpelelement(seed=2) # player_names=["Aaa", "Bbb", "Ccc", "Ddd"]) 
game.show_hands()

In [ ]:
# Zoek naar een eerste hand met twee A's
for seed in range(0, 100):
	game_x = SpelZonderSpelelement(seed=seed)
	kaarten = []
	for player in game_x.show_hands()[0]:
		kaarten.append(player[0])
	
	if kaarten.count('A') > 1:
		print(seed)

## Display

In [ ]:

# Create buttons
start_button = widgets.Button(description="Start Game", button_style="success", layout=widgets.Layout(width="150px"))
play_button = widgets.Button(description="Play Card", disabled=True, button_style="info", layout=widgets.Layout(width="150px"))  # Initially disabled
show_hands_button = widgets.Button(description="Show Hands", disabled=True, button_style="info", layout=widgets.Layout(width="150px"))  # Initially disabled

seed_input = widgets.IntText(value=2, description="Seed", style={'description_width': 'initial'})

output = widgets.Output()  # Apply CSS styling to the output

# Define a callback function to handle start button click
def on_start_button_click(b):
    global game
    game = SpelZonderSpelelement(seed=seed_input.value)  # Use the value from the seed input field
    with output:
        print(f"Game initialized successfully with Seed {seed_input.value}!")
    # Change the text of the start button to "Reset Game"
    start_button.description = "Reset Game"
    start_button.button_style = "danger"
    # Enable the play button and show hands button after the game is started
    play_button.disabled = False
    show_hands_button.disabled = False

# Define a callback function to handle play button click
def on_play_button_click(b):
    global game
    with output:
        game.play_card()  # Execute play_card() function
        print("\n")

# Define a callback function to handle show hands button click
def on_show_hands_button_click(b):
    global game
    with output:
        hands_data = game.show_hands()  # Get hands data
        # Create a Pandas DataFrame from the hands data
        df = pd.DataFrame(hands_data)
        # Display the DataFrame
        display(df)
        print("\n")

# Attach the callback functions to the button click events
start_button.on_click(on_start_button_click)
play_button.on_click(on_play_button_click)
show_hands_button.on_click(on_show_hands_button_click)

# Display the buttons and outpu
buttons_box = widgets.HBox([start_button, play_button, show_hands_button], layout=widgets.Layout(justify_content='space-around'))
widgets.VBox([seed_input, buttons_box, output])


## Simulaties

In [ ]:

game_x = SpelZonderSpelelement(seed=2)

aantal_spelers = 5
for ronde in range(200):
	print(f"Ronde {ronde}")
	game_x.play_card(print_output=True)
	_aantal_spelers = sum(game.show_hands()[0] == game.show_hands()[0])
	if aantal_spelers > _aantal_spelers:
		print(f"Ronde {ronde + 1} - Aantal spelers {_aantal_spelers}.")
	aantal_spelers = _aantal_spelers
	if aantal_spelers == 2:
		break	
print(f"Aantal rondes: {ronde + 1}")

In [ ]:
rounds_per_seed = []  # List to store (seed, rounds) tuples

for s in range(100):
    game_s = SpelZonderSpelelement(seed=s)
    aantal_spelers = 5
    rounds = 0

    for ronde in range(100):
        rounds += 1  # Increment rounds count for this seed
        game_s.play_card(print_output=False)
        _aantal_spelers = sum(game_s.show_hands()[0] == game_s.show_hands()[0])
        if aantal_spelers > _aantal_spelers:
            1
            # print(f"Ronde {ronde + 1} - Aantal spelers {_aantal_spelers}.")
        aantal_spelers = _aantal_spelers
        if aantal_spelers == 2:
            break

    rounds_per_seed.append((s, rounds))

# Create DataFrame
df = pd.DataFrame(rounds_per_seed, columns=['seed', 'number_of_rounds'])
df


In [ ]:
df["number_of_rounds"].idxmax()

In [ ]:
df["number_of_rounds"].hist()